This cell imports necessary modules for the simulation, including asyncio for asynchronous execution, yaml for configuration management, and logging for debugging. It also adds the project root to the Python path to ensure module imports work correctly.


In [6]:
import asyncio
import logging
import os
import sys
from pathlib import Path

import yaml

sys.path.append(os.path.abspath(".."))
# Add the project root to sys.path

from beam_3d_visualizer.beam_server.beam_visualization_wrapper import (
    BeamVisualizationWrapper,
)
from envs.beam_control_env import BeamControlEnv

from IPython.display import HTML, display

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

This function handles the main simulation loop. It initializes the environment, continuously updates the visualization, checks for new control actions, and logs step-wise simulation data. The loop runs until the simulation is complete.

**Expected Output:**

- Console messages indicating the start of the simulation and step-by-step log information such as received actions, computed rewards, and updated observations.
- If a WebSocket client is connected, visualization updates will be sent accordingly.


In [2]:
async def run_simulation(env: BeamControlEnv):
    """Run the simulation loop, stepping the environment with control actions."""
    print("\n--- Starting simulation loop ---")

    step_count = 0
    done = False

    # Reset the environment
    observation, _ = env.reset()

    while not done:
        # Render and broadcast data to clients
        # Calls BeamVisualizationWrapper.render(),
        # which delegates to WebSocketWrapper.render()
        await env.render()

        # Check if we have a new control action from WebSocket
        if env.control_action is not None:
            print(f"New action received: {env.control_action}")
            env.last_action = env.control_action  # Update last_action with new action
            env.control_action = None  # Clear after use

        # Step through the environment with the last action
        # Note: last_action persists and is reused if no new action is received
        observation, reward, terminated, truncated, info = env.step(env.last_action)

        # Update step count
        step_count += 1

        logger.info(
            "Step %d: Action = %s, Reward = %s, Observation = %s",
            step_count,
            env.last_action,
            reward,
            observation,
        )

        done = truncated  # or terminated

    env.close()
    print("Simulation completed.")

This function sets up the beam control environment by loading the configuration file (env_configs.yml) and initializing the simulation environment. It ensures the WebSocket-based visualization wrapper is correctly set up before running the simulation.

**Expected Output:**

- A message indicating that the WebSocket server is waiting for a client connection.
- Logging messages showing WebSocket and visualization wrapper initialization.
- A confirmation message once the simulation starts.


In [3]:
async def main():
    """Main entry point to set up the environment and start the simulation."""

    # Load configuration
    config_path = Path.home() / "Desktop" / "rl4aa25-tutorial" / "env_configs.yml"

    with config_path.open("r") as file:
        config = yaml.safe_load(file)

    # Initialize the environment and wrap it with WebSocketWrapper
    env = BeamControlEnv(config=config)
    env = BeamVisualizationWrapper(env)

    if not env.env.connected:
        print("Waiting for WebSocket client to connect...")
        await asyncio.sleep(5.0)  # Small delay to prevent CPU overload

    # Run the simulation
    await run_simulation(env)

    print("Simulation shutdown completed.")

This cell asynchronously starts the simulation in the background and allows users to run other cells without blocking execution. A short delay is added to ensure WebSocket services are properly started.

**Expected Output:**

- Log messages confirming WebSocket server startup and connection establishment.

- If successful, the WebSocket client (JavaScript-based visualization) connects, and the event display (3D visualization of the beamline) updates accordingly.


In [ ]:
# Create a background task for the simulation
simulation_task = asyncio.create_task(main())

# Add a small delay to allow the web server and WebSocket to start
await asyncio.sleep(2.0)  # Adjust delay as needed

print("Simulation is running in the background. You can now run other cells.")

## Expected Visualization Output

- A real-time 3D visualization of the beamline appears in the web application.
- A control panel enables users to adjust magnet strengths and steering angles, updating the particle beam trajectory dynamically.
- A scatter plot of particle positions is displayed, showing the beam’s distribution in real-time.


In [5]:
# Define the iframe as a string
iframe = '<iframe src="http://localhost:5173/" title="3D Lattice Visualization" height="600" width="1000"></iframe>'

display(HTML(iframe))

/Users/christiancontrerascampana/anaconda3/envs/rlaa2025/lib/python3.11/site-packages/IPython/core/display.py:447: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
